In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
holidays_events = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv')
transactions = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')

**An important function that can be used. Shows everything about a dataset in one go.**

In [3]:
def check_data(df):
    print(df.head())
    print(df.info())
    print(df.shape)

In [4]:
check_data(oil)

In [5]:
check_data(holidays_events)

In [6]:
check_data(stores)

In [7]:
check_data(transactions)

In [8]:
check_data(train)

In [9]:
holidays_events['date'] = pd.to_datetime(holidays_events['date'])
oil['date'] = pd.to_datetime(oil['date'])
transactions['date'] = pd.to_datetime(transactions['date'])
train['date'] = pd.to_datetime(train['date'])

In [10]:
oil.rename(columns={'dcoilwtico':'oil_price'}, inplace=True)

In [11]:
holidays_events["type"]=holidays_events["type"].astype("category")
stores["cluster"]=stores.cluster.astype("category")
stores["type"]=stores.type.astype("category")
train.family=train.family.astype('category')
train.store_nbr= train.store_nbr.astype('category')

In [12]:
df_merged = train.merge(holidays_events, on = 'date', how = 'left')
df_merged = df_merged.merge(oil, on = 'date', how = 'left')
df_merged = df_merged.merge(stores, on = 'store_nbr', how='left')
df_merged = df_merged.merge(transactions, on = ['date', 'store_nbr'], how='left')

In [13]:
df_merged.head()

In [14]:
df_merged = df_merged.rename(columns = {"type_x" : "holiday_type", "type_y" : "store_type"})
df_merged['date'] = pd.to_datetime(df_merged['date'])
df_merged['year'] = df_merged['date'].dt.year
df_merged['month'] = df_merged['date'].dt.month
df_merged['week'] = df_merged['date'].dt.isocalendar().week
df_merged['day'] = df_merged['date'].dt.day
df_merged['quarter'] = df_merged['date'].dt.quarter
df_merged['day_of_week'] = df_merged['date'].dt.day_name()

In [15]:
df_merged.head()

In [16]:
df_merged.info()

In [17]:
# Making a dataframe grouping the year and sales in order to plot it soon
df_y_s = df_merged.groupby('year').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)
df_y_s['sales'] = round(df_y_s['sales'],2)

# Now creating one for the days of the week
df_dw_s = df_merged.groupby('day_of_week').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)
df_dw_s['sales'] = round(df_dw_s['sales'],2)

# For months
df_m_s = df_merged.groupby('month').agg({"sales" : "mean"}).reset_index()
df_m_s['sales'] = round(df_m_s['sales'],2)
df_m_s['month'] =['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

# Quarters
df_q_s = df_merged.groupby('quarter').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)[:10]
df_q_s['sales'] = round(df_q_s['sales'],2)

In [18]:
# Setting the colors I'm going to use
df_y_s['color'] =["#458cff","#63a7ff", "#82c2ff","#82c2ff","#a0dcff"]
df_m_s['color'] = ["#00876c","#3d9a70","#64ad73","#89bf77","#afd17c","#d6e184","#fff18f","#fdd576","#fbb862","#f59b56","#ee7d4f","#e35e4e"]
df_dw_s['color'] = ["#d43d51","#e6785c","#f3aa79", "#ffd9a6","#f4cc93","#e8c181","#dcb56e"]


In [19]:
import calendar
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.offline as offline
import plotly.graph_objs as go

In [20]:
# Creating subplots so we can have many graphs in the same image
fig = make_subplots(rows=2, cols=2, 
                    specs=[[{"type": "bar"},{"type": "bar"} ],
                           [{"type": "bar"}, {"type": "pie"}]],
                    subplot_titles=("Avg Sales by Year", 
                                    "Avg Sales by Day of the Week", 
                                    "Avg Sales by Month", 
                                    "Avg Sales by Quarter"))
# Adding the charts I want to analyze
fig.add_trace(go.Pie(values=df_q_s['sales'], 
                     labels=df_q_s['quarter'], 
                     name='Quarter',
                     marker=dict(colors=["#0e455d", "#4f647d", "#8a8ba8", "#764c05"], 
                     line=dict(color='#000000', width=2)), 
                     hole=0,
                     hoverinfo='label+percent+value', 
                     textinfo='label'), 
                    row=2, col=2)
fig.update_traces(hovertemplate=None, 
                  textposition='outside',
                  textinfo='percent+label', 
                  rotation=0)
fig.append_trace(go.Bar(x=df_y_s['year'], 
                        y=df_y_s['sales'],
                        textposition='auto',
                        text=df_y_s['sales'], 
                        marker = {'color': list(df_y_s['color'])}),
                 row=1, col=1)
fig.add_trace(go.Bar(x=df_m_s['month'], y=df_m_s['sales'], 
                     marker=dict(color= df_m_s['color']),
                     text=df_m_s['sales'],
                     textposition='auto',
                     name='Month'), 
                     row=2, col=1)
fig.append_trace(go.Bar(x=df_dw_s['day_of_week'], 
                        y=df_dw_s['sales'], 
                        textposition='auto',
                        text=df_dw_s['sales'],
                        marker = {'color': list(df_dw_s['color'])}), 
                 row=1, col=2)


fig.update_layout(height=1000, 
                  width=1400, 
                  title_text="AVG SALES",  
                  title_font=dict(size=40, color='#8a8d93'), 
                  showlegend=False)

fig.show()

In [21]:
# Creating a dataframe with the city and sales
df_c_s = df_merged.groupby('city').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)
df_c_s['sales'] = round(df_c_s['sales'],2)

# Now for the state
df_st_s = df_merged.groupby('state').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)
df_st_s['sales'] = round(df_st_s['sales'],2)

# Product family
df_f_s = df_merged.groupby('family').agg({"sales" : "mean"}).reset_index().sort_values(by='sales', ascending=False)[:10]
df_f_s['sales'] = round(df_f_s['sales'],2)

# For oil in order to see if oil price affects sales or not
df_o_s = df_merged.groupby('oil_price').agg({"sales" : "mean"}).reset_index()
df_o_s['sales'] = round(df_o_s['sales'],2)

sizeref = 2.*max(df_o_s['sales'])/(30**2)

In [22]:
# Colors for the graphs
df_c_s['color'] = ["#440154","#471365","#482374","#46327e","#424086","#3d4e8a","#365c8d","#31688e","#2c738e","#277f8e","#238a8d","#1f968b","#1fa187","#26ad81","#35b779","#4ac16d","#65cb5e","#81d34d","#a0da39","#c0df25","#dfe318","#fde725"]
df_st_s['color'] = ["#002051","#022c65","#14386d","#2b446e","#42506e","#575c6e","#696970","#787573","#868276","#948f78","#a49d78","#b6ab73","#caba6a","#e0c95d","#f2d950","#fdea45"]
df_f_s['color'] = ["#0b090a","#161a1d","#660708","#a4161a","#ba181b","#e5383b","#b1a7a6","#d3d3d3","#f5f3f4","#ffffff"]

In [23]:
fig = make_subplots(rows=2, cols=2, 
                    specs=[[{"type": "bar"},{"type": "bar"} ],
                           [{"type": "bar"}, {"type": "scatter"}]],
                    subplot_titles=("Avg Sales by State", 
                                    "Avg Sales by City", 
                                    "Avg Sales by Product Family", 
                                    "Sales by Oil Price"))


fig.append_trace(go.Bar(x=df_st_s['state'], 
                        y=df_st_s['sales'],
                        textposition='auto',
                        text=df_st_s['sales'], 
                        marker = {'color': list(df_st_s['color'])}),
                row=1, 
                col=1)
fig.append_trace(go.Bar(x=df_c_s['city'], 
                        y=df_c_s['sales'],
                        textposition='auto',
                        text=df_c_s['sales'], 
                        marker = {'color': list(df_c_s['color'])}),
                row=1, 
                col=2)
fig.append_trace(go.Bar(x=df_f_s['family'], 
                        y=df_f_s['sales'],
                        textposition='auto',
                        text=df_f_s['sales'], 
                        marker = {'color': list(df_f_s['color'])}),
                row=2, 
                col=1)
fig.append_trace(go.Scatter(x = df_o_s['oil_price'], 
                            y = df_o_s['sales'],  
                            mode = 'markers' , 
                            hovertemplate="<b>Oil_Price: %{x}<br><br><br>" + "<b>Sales: %{y}<br><br><br>", 
                            marker = dict(color = df_o_s['sales'],
                            colorscale = 'portland',
                            opacity = 0.8,
                            sizemode = 'area', 
                            sizeref= sizeref,
                            sizemin= 4, showscale = False)), 
                 row=2, 
                 col=2)

fig.update_xaxes(title_text="Oil_Price", 
                 row=2, 
                 col=2)
fig.update_yaxes(title_text="Sales", 
                 row=2, 
                 col=2)

fig.update_layout(height=1000, 
                  width=1400, 
                  title_text="AVG SALES 2",  
                  title_font=dict(size=40, color='#8a8d93'), 
                  showlegend=False)

fig.show()

In [24]:
# Create a dataframe for city sales to assist in our plotting
df_cs_s = df_merged.groupby('city').agg({"sales" : "sum"}).reset_index().sort_values(by='sales', ascending=False)
df_cs_s['sales'] = round(df_cs_s['sales'],2)

# Now for the stores
df_store_s = df_merged.groupby(['city', 'store_type']).agg({"store_nbr" : "nunique"}).reset_index().sort_values(by='store_nbr', ascending=False)

# Now for the promotions
df_op_s = df_merged.groupby(['city']).agg(sales_mean = ('sales', 'mean'), onpromotion_mean = ('onpromotion', 'mean')).reset_index().sort_values(by='sales_mean', ascending=False)
df_op_s['sales_mean'] = round(df_op_s['sales_mean'],2)
sizeref2 = 2.*max(df_op_s['sales_mean'])*100


In [25]:
fig = make_subplots(rows=2, cols=2, 
                    specs=[[{"type": "bar"},{"type": "bar"} ],
                           [{"type": "bar"}, {"type": "scatter"}]],
                    subplot_titles=("Total Sales by State", 
                                    "Total stores in City", 
                                    "Average Items on Promotion"))


fig.append_trace(go.Bar(x=df_cs_s['city'], 
                        y=df_cs_s['sales'],
                        textposition='auto',
                        text=df_cs_s['sales'], 
                        marker = {'color': list(df_c_s['color'])}),
                row=1, 
                col=1)
fig.append_trace(go.Bar(x=df_store_s['city'], 
                        y=df_store_s['store_nbr'],
                        textposition='auto',
                        text=df_store_s['store_nbr'], 
                        marker = {'color': list(df_st_s['color'])}),
                row=1,
                col=2)
fig.append_trace(go.Scatter(x = df_op_s['sales_mean'], 
                            y = df_op_s['onpromotion_mean'],  
                            mode = 'markers+text' , 
                            text = df_op_s['city'],
                            textposition='top center', 
                            marker = dict(color = df_op_s['sales_mean'],
                            colorscale = 'YlOrRd',
                            opacity = 0.8,
                            sizemode = 'area', 
                            size = df_op_s['sales_mean'],
                            sizemin= 4, 
                            showscale = False)), 
                 row=2, 
                 col=1)
fig.update_layout(template="ggplot2",
                  height=1000, 
                  width=1400, 
                  title_text="Total Sales by State, Store numbers and Average Items on Promotion",  
                  title_font=dict(size=40, color='#8a8d93'), 
                  showlegend=False)


fig.show()